In [1]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

In [2]:
import torch
import torch.nn as nn

In [3]:
from soft_embedding import SoftEmbedding

n_tokens = 20
initialize_from_vocab = True

In [4]:
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [5]:
s_wte = SoftEmbedding(model.get_input_embeddings(), 
                      n_tokens=n_tokens, 
                      initialize_from_vocab=False)

In [6]:
model.set_input_embeddings(s_wte)

In [7]:
inputs = tokenizer("May the force be", return_tensors="pt")

# need to pad attention_mask and input_ids to be full seq_len + n_learned_tokens
# even though it does not matter what you pad input_ids with, it's just to make HF happy
inputs['input_ids'] = torch.cat([torch.full((1,n_tokens), 50256), inputs['input_ids']], 1)
inputs['attention_mask'] = torch.cat([torch.full((1,n_tokens), 1), inputs['attention_mask']], 1)

outputs = model(**inputs)